In [1]:
pip install --upgrade --quiet langchain_openai langchain_core psycopg2

DEPRECATION: Loading egg at /Users/ashokarulsamy/anaconda3/lib/python3.11/site-packages/basicsr-1.4.2-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at /Users/ashokarulsamy/anaconda3/lib/python3.11/site-packages/facexlib-0.3.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at /Users/ashokarulsamy/anaconda3/lib/python3.11/site-packages/opencv_python-4.11.0.86-py3.11-macosx-11.1-arm64.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at /Users/ashokarulsamy/anaconda3/lib/python3.11/site-packages/gfpgan-1.3.8-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg

In [ ]:
# graph.query("""
#     UNWIND ['Tom Cruise', 'Val Kilmer', 'Anthony Edwards', 'Meg Ryan'] AS actor
#     MERGE (m:Movie {title:'Top Gun'})
#     MERGE (a:Actor {name: actor})
#     MERGE (a)-[:ACTED_IN]->(m)
# """)
# )

In [2]:
#read data from graph db and print

import psycopg2

# Database connection parameters
DB_HOST = "localhost"
DB_PORT = "5455"
DB_NAME = "postgresDB"
DB_USER = "postgresUser"
DB_PASSWORD = "postgresPW"
GRAPH_NAME = "age_test"

# Connect to PostgreSQL
conn = psycopg2.connect(
    host=DB_HOST,
    port=DB_PORT,
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD
)

def run_cypher_query(query: str):
    """Execute a Cypher query against Apache AGE"""
    with conn.cursor() as cur:
        # Ensure AGE is loaded in session
        cur.execute("LOAD 'age';")
        cur.execute("SET search_path = ag_catalog, public;")

        # Format Cypher query
        cypher_sql = f"""
        SELECT * FROM cypher('{GRAPH_NAME}', $$ {query} $$) AS (result agtype);
        """

        # Execute query
        cur.execute(cypher_sql)

        # Fetch all results
        rows = cur.fetchall()
        return rows



In [3]:
#read data from graph db and print
if __name__ == "__main__":
    cypher_query = "MATCH (n) RETURN n LIMIT 5"
    result = run_cypher_query(cypher_query)
    for row in result:
        print(row)


('{"id": 844424930131969, "label": "Movie", "properties": {"title": "Top Gun"}}::vertex',)
('{"id": 844424930131970, "label": "Movie", "properties": {"title": "Top Gun"}}::vertex',)
('{"id": 844424930131971, "label": "Movie", "properties": {"title": "Top Gun"}}::vertex',)
('{"id": 844424930131972, "label": "Movie", "properties": {"title": "Top Gun"}}::vertex',)
('{"id": 1125899906842625, "label": "Actor", "properties": {"name": "Tom Cruise"}}::vertex',)


In [4]:
#execute natural language query against graph db using llm
import psycopg2
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import Runnable

# OpenAI LLM
llm = ChatOpenAI(temperature=0, openai_api_key="")

# Apache AGE Cypher prompt
cypher_prompt_template = PromptTemplate(
    input_variables=["query"],
    template="""
You are a Cypher expert for the Apache AGE graph database.
Given a user question, generate a Cypher query that runs on a graph named 'age_test'.

Question: {query}

Cypher Query:
"""
)


# Connect to PostgreSQL/AGE
def run_cypher_on_age(cypher_query: str):
    conn = psycopg2.connect(
        host="localhost",
        port="5455",
        dbname="postgresDB",
        user="postgresUser",
        password="postgresPW"
    )
    with conn.cursor() as cur:
        cur.execute("LOAD 'age';")
        cur.execute("SET search_path = ag_catalog, public;")
        sql = f"SELECT * FROM cypher('age_test', $$ {cypher_query} $$) AS (result agtype);"
        cur.execute(sql)
        rows = cur.fetchall()
    conn.close()
    return rows

# Runnable chain
class AGEGraphQAChain(Runnable):
    def __init__(self, llm, prompt_template):
        self.llm = llm
        self.prompt_template = prompt_template

    def invoke(self, query: str):
        # 1. Generate Cypher query using LLM
        cypher_prompt = self.prompt_template.format(query=query)
        cypher_query = self.llm.invoke(cypher_prompt).content.strip()
        print("Generated Cypher:\n", cypher_query)

        # 2. Execute Cypher query
        try:
            results = run_cypher_on_age(cypher_query)
            return {"cypher": cypher_query, "results": results}
        except Exception as e:
            return {"cypher": cypher_query, "error": str(e)}

# Example usage
if __name__ == "__main__":
    qa_chain = AGEGraphQAChain(llm, cypher_prompt_template)
    question = "List all actors who acted in Top Gun"
    response = qa_chain.invoke(question)

    print("\nResults:")
    print(response)


Generated Cypher:
 MATCH (a:Actor)-[:ACTED_IN]->(m:Movie {title: 'Top Gun'})
RETURN a.name

Results:
{'cypher': "MATCH (a:Actor)-[:ACTED_IN]->(m:Movie {title: 'Top Gun'})\nRETURN a.name", 'results': [('"Tom Cruise"',), ('"Val Kilmer"',), ('"Anthony Edwards"',), ('"Meg Ryan"',)]}


In [ ]:
pip install --upgrade --force-reinstall numpy pandas # only if any compatibility issue with panda and numpy

In [10]:
#execute natural language query against graph db using llm
import psycopg2
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import Runnable

# OpenAI LLM
llm = ChatOpenAI(temperature=0, openai_api_key="")

# Apache AGE Cypher prompt
cypher_prompt_template = PromptTemplate(
    input_variables=["query"],
    template="""
You are a Cypher expert for the Apache AGE graph database.
Given a user question, generate a Cypher query that runs on a graph named 'age_test'.

Question: {query}

Cypher Query:
"""
)


# Connect to PostgreSQL/AGE
def run_cypher_on_age(cypher_query: str):
    conn = psycopg2.connect(
        host="localhost",
        port="5455",
        dbname="postgresDB",
        user="postgresUser",
        password="postgresPW"
    )
    with conn.cursor() as cur:
        cur.execute("LOAD 'age';")
        cur.execute("SET search_path = ag_catalog, public;")
        sql = f"SELECT * FROM cypher('cmdb_graph', $$ {cypher_query} $$) AS (result agtype);"
        cur.execute(sql)
        rows = cur.fetchall()
    conn.close()
    return rows

# Runnable chain
class AGEGraphQAChain(Runnable):
    def __init__(self, llm, prompt_template):
        self.llm = llm
        self.prompt_template = prompt_template

    def invoke(self, query: str):
        # 1. Generate Cypher query using LLM
        cypher_prompt = self.prompt_template.format(query=query)
#         cypher_query = self.llm.invoke(cypher_prompt).content.strip()
        cypher_query = self.llm.invoke(cypher_prompt).content.strip().rstrip(';')
        print("Generated Cypher:\n", cypher_query)

        # 2. Execute Cypher query
        try:
            results = run_cypher_on_age(cypher_query)
            return {"cypher": cypher_query, "results": results}
        except Exception as e:
            return {"cypher": cypher_query, "error": str(e)}

# Example usage
if __name__ == "__main__":
    qa_chain = AGEGraphQAChain(llm, cypher_prompt_template)
    question = "list all the relationship for database SB1"
    response = qa_chain.invoke(question)

    print("\nResults:")
    print(response)


Generated Cypher:
 MATCH (n:Database {name: 'SB1'})-[r]->()
RETURN r

Results:
{'cypher': "MATCH (n:Database {name: 'SB1'})-[r]->()\nRETURN r", 'results': []}


In [11]:
#read data from graph db and print

import psycopg2

# Database connection parameters
DB_HOST = "localhost"
DB_PORT = "5455"
DB_NAME = "postgresDB"
DB_USER = "postgresUser"
DB_PASSWORD = "postgresPW"
GRAPH_NAME = "cmdb_graph"

# Connect to PostgreSQL
conn = psycopg2.connect(
    host=DB_HOST,
    port=DB_PORT,
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD
)

def run_cypher_query(query: str):
    """Execute a Cypher query against Apache AGE"""
    with conn.cursor() as cur:
        # Ensure AGE is loaded in session
        cur.execute("LOAD 'age';")
        cur.execute("SET search_path = ag_catalog, public;")

        # Format Cypher query
        cypher_sql = f"""
        SELECT * FROM cypher('{GRAPH_NAME}', $$ {query} $$) AS (result agtype);
        """

        # Execute query
        cur.execute(cypher_sql)

        # Fetch all results
        rows = cur.fetchall()
        return rows

#read data from graph db and print
if __name__ == "__main__":
    cypher_query = "MATCH (n) RETURN n LIMIT 5"
    result = run_cypher_query(cypher_query)
    for row in result:
        print(row)

('{"id": 844424930131969, "label": "Server", "properties": {"name": "AppServer1"}}::vertex',)
('{"id": 844424930131970, "label": "Server", "properties": {"name": "DBServer1"}}::vertex',)
('{"id": 1125899906842625, "label": "Application", "properties": {"name": "App1"}}::vertex',)
('{"id": 1407374883553281, "label": "Database", "properties": {"name": "DB1"}}::vertex',)
('{"id": 1688849860263937, "label": "Network", "properties": {"name": "Subnet1"}}::vertex',)


In [13]:
# Example usage
if __name__ == "__main__":
    qa_chain = AGEGraphQAChain(llm, cypher_prompt_template)
    question = "Subnet1"
    response = qa_chain.invoke(question)

    print("\nResults:")
    print(response)


Generated Cypher:
 MATCH (n:Subnet {name: 'Subnet1'}) RETURN n

Results:
{'cypher': "MATCH (n:Subnet {name: 'Subnet1'}) RETURN n", 'results': []}
